In [ ]:
#Fuzzy BoW - create and search through bag-of-words generated from transcriptions
##Transcriptions generated from HandPrint(https://github.com/caltechlibrary/handprint)
##Generates bag-of-words from individual image transcriptions
import os
import sys
import re
from pathlib import Path

#input/output
paths = '....txt'#plain text list of urls or filenames (pre-existing)
target = '...' #HP outputs (pre-existing)
textOut = '....txt'#Bag-of-Words output

#declarations
BoW = open(textOut, "a")

#append bag-of-words with headers and transcriptions
for path in sorted(Path(target).rglob('*.txt')):
    header = path.stem.split('.')
    BoW.write("\n")
    BoW.write(header[0]) #write headers location to bag-of-words
    BoW.write("\n")
    print((header[0]) + " added to bag-of-words")
    absolute = (str(path.parent) + "/" + path.name)
    contents = open(absolute, "r") 
    BoW.write(str(contents.read())) #write transcriptions to bag-of-words
    print(("Transcription for " + header[0]) + " added to bag-of-words")
    BoW.write("\n")
BoW.close()
print("Bag-of-words generated and output from corpus")


In [ ]:
###Fuzzy Search Bag-of-Words
###From "fuzzy-search" by Marijn Koolen (https://github.com/marijnkoolen/fuzzy-search)
##Matt Cook - 2021

from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher
from fuzzy_search.fuzzy_phrase_model import PhraseModel
import json
import re
from pathlib import Path

#declarations
text = ".../BOW_1838-1869_MSFT.txt"#bag of words, output from above
target = '.../Lag 1838-1869_MSFT' #HandPrint outputs (pre-existing)
searchTerms = open("...Terms.txt", "r") #list of terms to be searched in the bag-of-words
variants = [] #list of search term variations
passages = open("...excerpts.txt", "w") #output document

#user input
terms = searchTerms.read()
terms = terms.split("\n")
print("Searching bag-of-words for the following: " + (str(terms).strip("''][")))
passages.write("Fuzzy search completed for the following keywords: " + (str(terms)))
passages.write("\n\n")
print("\n")
counter = 0

#search parameters
##see: https://github.com/marijnkoolen/fuzzy-search/blob/master/fuzzy_search/fuzzy_phrase_searcher.py
config = {
    #higher thresholds when there are few OCR errors
    'char_match_threshold': 0.8,
    'ngram_threshold': 0.7,
    'skipgram_threshold': 0.7,
    'levenshtein_threshold': 0.5,
    "skip_exact_matching": False,
    "punctuation": [" ", "\n"],
    'include_variants': True, # for phrases that have variant phrasings
    'filter_distractors': False, # avoid matching with similar but different phrases
    "ignorecase": True, # Is upper/lowercase a meaningful signal?
    "use_word_boundaries": True,# should matches follow word boundaries?
    "max_length_variance": 2, # matching string can be lower/shorter than prhase
}

# initialize a new searcher instance with the config
fuzzy_searcher = FuzzyPhraseSearcher(config)
phrase_model = PhraseModel(phrases=terms)
fuzzy_searcher.index_phrase_model(phrase_model)

#identify matches in the text using fuzzy search package
BoW = open(text, "r")
for match in fuzzy_searcher.find_matches(BoW.read()):
    variant = match.json()
    variant = variant['string']
    variants.append(variant)
passages.write("\n\n")
print("\n")
print((str(len(variants))) + " relevant variations detected, including: ")
print(variants)
passages.write((str(len(variants))) + " relevant variations detected, including: ")
passages.write("\n")
passages.write(str(variants))
passages.write("\n\n")
print("\n")
BoW.close()

#Write specific records that include found variants
passages.write("Variants of input search terms appear on the following records: ")
passages.write("\n\n")
print("variants appear on the following records:")
print("\n")
for path in sorted(Path(target).rglob('*.txt')):
    header = path.stem.split('.')
    absolute = (str(path.parent) + "/" + path.name)
    contents = open(absolute, "r")
    contents = contents.read()
    for variant in variants:
        if variant in contents:
            passages.write(header[0] + "\n" + contents)
            passages.write("\n\n")
            print("Record: " + header[0] + "\n" + "Transcription: " + contents)
            print("\n")
passages.close()

print("Matched records saved to disk")
print("Have a nice day")
        